# GARCHコードサンプル

結論から言うと、

* コードの流れと GARCH(1,1) の使い方としては「ほぼ正しい」です
* ただし **説明コメントの一部（`alpha[2]`, `beta[2]`）が間違い** になっているのと、
* 予測ボラティリティのプロットの軸・日付周りを少し整えると、教材としてかなりきれいになります

なので、

1. 修正版の「完成コード」
2. そのコードのパートごとの詳しい解説
   を順に書きます。

---

## 1. 修正版フルコード（コメント付き）

```python
# ===========================
# 0. 必要なライブラリのインストール（初回のみ）
# ===========================
# !pip install arch yfinance

# ===========================
# 1. ライブラリのインポート
# ===========================
import pandas as pd
import numpy as np
import yfinance as yf
from arch import arch_model
import matplotlib.pyplot as plt
from statsmodels.stats.diagnostic import acorr_ljungbox
from pandas.tseries.offsets import BDay  # 予測ボラティリティ用の日付生成に使用

# ===========================
# 2. データ取得 & 日次リターンの計算
# ===========================
# S&P 500 (^GSPC) の日次データを取得
data = yf.download('^GSPC', start='2010-01-01', end='2023-12-31')

# 調整終値のパーセンテージ変化（リターン）を計算
# 通常 pct_change() は 0.01 = 1% のような「小さい値」になります。
# GARCH 推定では、最適化を安定させるために 100倍して「%単位」にするのが
# 実務でもよく推奨されています（例: 0.5% → 0.5）:contentReference[oaicite:0]{index=0}
returns = 100 * data['Adj Close'].pct_change().dropna()
returns.name = 'sp500_returns'

# ===========================
# 3. リターン系列の可視化（ボラティリティ・クラスタリングの確認）
# ===========================
plt.figure(figsize=(12, 6))
returns.plot(title='S&P 500 Daily Returns (in %)')
plt.ylabel('Daily Return (%)')
plt.grid(True)
plt.show()

# ===========================
# 4. GARCH(1,1) モデルの指定と推定
# ===========================
# arch_model の主な引数:
# - mean='Constant' : r_t = μ + ε_t という「定数平均」モデル
# - vol='Garch'     : 分散方程式に GARCH(p, q) を使う （大小文字は 'Garch' でOK）:contentReference[oaicite:1]{index=1}
# - p=1, q=1        : GARCH(1,1) モデル
# - dist='Normal'   : 残差 z_t ~ N(0,1) を仮定
am = arch_model(
    returns,
    mean='Constant',
    vol='Garch',
    p=1,
    q=1,
    dist='Normal'
)

# モデルの推定
# update_freq=5 : 5イテレーションごとに推定の進捗を表示
result = am.fit(update_freq=5)

# 推定結果のサマリーを表示
print(result.summary())

# ===========================
# 5. 係数の解釈メモ（説明用）
# ===========================
"""
係数の典型的な読み方（例）:
  mu         : 平均リターン μ
  omega      : 分散方程式の定数項 ω
  alpha[1]   : 直近のショック ε_{t-1}^2 が分散に与える影響（ARCH 項）
  beta[1]    : 直近の分散 σ_{t-1}^2 が分散に与える影響（GARCH 項）

もし alpha[1] = 0.10, beta[1] = 0.88 と推定された場合:
  ・持続性: α_1 + β_1 = 0.98 となり、1に非常に近い
      → 一度大きなショックが起きると、その影響が長く残り、
        ボラティリティの高い状態がしばらく続くことを意味する
  ・有意性: alpha[1], beta[1] の P>|z| がどちらも 0.05 より十分小さいか確認
      → 小さいほど「この係数は 0 ではない」と統計的に言える
      → もし p値が大きい（例: 0.3）なら、その係数は統計的に有意でない
       （＝そのラグは効いていない可能性があり、次数やモデルの見直し候補）
"""

# ===========================
# 6. モデル診断：標準化残差と Ljung-Box 検定
# ===========================
# 標準化残差: \tilde{ε}_t = ε_t / σ_t
std_resid = result.std_resid.dropna()

print("\n--- GARCH Model Diagnostics ---")

# (1) 標準化残差そのものの Ljung-Box 検定
#     → 平均モデル（ここでは定数 + GARCH）の残差に自己相関が残っていないか？
lb_test_resid = acorr_ljungbox(std_resid, lags=[1, 5, 10], return_df=True)
print("\nLjung-Box test on Standardized Residuals:")
print(lb_test_resid)
print("※ lb_pvalue がすべて 0.05 より大きいことが望ましい（自己相関なし）")

# (2) 標準化残差の2乗の Ljung-Box 検定
#     → 条件付き分散モデル（GARCH）が ARCH 効果を取り切れているか？
lb_test_sq_resid = acorr_ljungbox(std_resid**2, lags=[1, 5, 10], return_df=True)
print("\nLjung-Box test on Squared Standardized Residuals:")
print(lb_test_sq_resid)
print("※ lb_pvalue がすべて 0.05 より大きいことが望ましい（残りの ARCH 効果なし）")

"""
診断のざっくりした読み方:
  - (1) 標準化残差の Ljung-Box 検定で p値が小さい:
       → 平均モデル（mean='Constant'）が不十分で、まだ予測可能なパターンが残っている可能性
         mean='AR' など AR-GARCH モデルを検討
  - (2) 標準化残差^2 の Ljung-Box 検定で p値が小さい:
       → GARCH(1,1) でもボラティリティクラスタリングを取り切れていない
         → GARCH(1,2), GARCH(2,1) や EGARCH, GJR-GARCH などの拡張を検討
  - 両方とも十分大きな p値:
       → この GARCH(1,1) は「指定としては大きな問題はなさそう」と判断でき、
         ボラティリティ予測に進んでよい
"""

# ===========================
# 7. 将来 5日間のボラティリティ予測
# ===========================
# horizon=5 で 5ステップ先までの条件付き分散 σ_t^2 を予測
forecasts = result.forecast(horizon=5)

# forecasts.variance は DataFrame:
#   行: 過去データの日付
#   列: 予測ホライズン (h.1, h.2, ..., h.5)
# 最後の行 (iloc[-1]) が「サンプルの最終日から見た」将来5日分の予測分散
predicted_variance = forecasts.variance.iloc[-1]
print("\n--- Predicted Variance (5 days ahead) ---")
print(predicted_variance)

# 分散の平方根をとって「標準偏差（ボラティリティ）」に変換
predicted_volatility = np.sqrt(predicted_variance)

print("\n--- Predicted Volatility (Std. Dev. in %) ---")
print(predicted_volatility)
print("※ リターンを 100 倍しているので、これは『日次リターンの%ボラティリティ』です。")

# ===========================
# 8. 過去のフィットされたボラティリティと将来予測の可視化
# ===========================
plt.figure(figsize=(12, 6))

# 過去の条件付きボラティリティ（σ_t）
plt.plot(
    result.conditional_volatility,
    label='Fitted GARCH(1,1) Volatility'
)

# 予測ボラティリティ用に「将来の日付インデックス」を作る
last_date = returns.index[-1]
future_index = [last_date + BDay(i) for i in range(1, len(predicted_volatility) + 1)]
predicted_volatility.index = future_index

# 将来の予測ボラティリティ
plt.plot(
    predicted_volatility,
    'r--',
    label='Forecasted Volatility (Next 5 Business Days)'
)

plt.title('GARCH(1,1) Fitted and Forecasted Volatility (S&P 500)')
plt.ylabel('Volatility (Std. Dev. in %)')
plt.legend()
plt.grid(True)
plt.show()
```

---

## 2. どこが「間違い／改善ポイント」だったか

### (1) `alpha[2]`, `beta[2]` の説明

元コードのコメント：

```python
"""
もし alpha[2] が 0.1、beta[2] が 0.88 と推定された場合：
...
"""
```

* GARCH(1,1) では **αのラグは1つだけ** なので、`alpha[1]`, `beta[1]` が正しいです
* `result.summary()` を出してみると、実際に列名は `alpha[1]`, `beta[1]` になっています ([Qiita][1])

→ ここだけ **添字を 1 に修正** しました。

---

### (2) Ljung-Box 検定のラグ

元コードでは：

```python
lb_test_resid = acorr_ljungbox(std_resid, lags=[1], return_df=True)
...
lb_test_sq_resid = acorr_ljungbox(std_resid**2, lags=[1], return_df=True)
```

でも良いのですが、教材としては

* ラグ1だけでなく、**1, 5, 10 など複数のラグ**で p値を見た方が、
  「どのタイムスケールで自己相関が残っているか」が見えてわかりやすいです。([Read the Docs][2])

→ `lags=[1, 5, 10]` に変更しました（もちろん、実務では 10, 20 など好きに変えてOK）。

---

### (3) ボラティリティ予測のプロットの横軸

元コードでは：

```python
plt.plot(result.conditional_volatility, label='Fitted ...')
plt.plot(predicted_volatility, 'r--', label='Forecasted ...')
```

* `result.conditional_volatility` は「過去日付の index」を持った Series
* `predicted_volatility` は「h.1, h.2, ...」の index（将来日付ではない）

になっており、**横軸のスケールが混ざっている**状態になります。

→ 修正版では、`pandas.tseries.offsets.BDay` を使い、

* 「最後の日付 + 1営業日, ... + 5営業日」の index を作って
* `predicted_volatility.index = future_index` としてからプロット

するようにしました。これで、グラフの横軸が「日付として自然な連続」になります。

---

## 3. 各ステップの「GARCH 的な意味」

最後に、GARCH モデルを「どう使うか」という視点で、コードの流れをまとめます。

### ① リターンを作って 100 倍する理由

```python
returns = 100 * data['Adj Close'].pct_change().dropna()
```

* GARCH は「**リターンの条件付き分散**」をモデル化するので、
  **価格ではなくリターン**（%変化）の系列を使います。
* arch のドキュメントでも、`returns = 100 * price.pct_change()` のような形が推奨されています。([arch.readthedocs.io][3])

  * スケールが小さすぎると最適化が不安定になりやすい
  * 100倍して「%単位」にすると、他のパラメータとスケールが揃って推定が安定

### ② `arch_model` で GARCH(1,1) を指定

```python
am = arch_model(
    returns,
    mean='Constant',
    vol='Garch',
    p=1,
    q=1,
    dist='Normal'
)
```

* **平均方程式**:
  $( r_t = \mu + \epsilon_t )$

* **分散方程式（GARCH(1,1)）**: ([ウィキペディア][4])

$$
  \sigma_t^2 = \omega + \alpha_1 \epsilon_{t-1}^2 + \beta_1 \sigma_{t-1}^2
$$

* これで、「直近のショック（ε²）」と「直近の分散」が、
  どれくらい次の分散に効いているか（α, β）を推定できます。

### ③ サマリーで何を見るか

`result.summary()` を見ると、主にチェックしたいのは：

* `mu`：平均リターン（金融では 0 に近いことも多い）
* `omega`：平常時の「ベース」分散レベル
* `alpha[1]`：ショックへの即時反応
* `beta[1]`：ボラティリティの持続性
* `alpha[1] + beta[1]` が 1 に近い → ショックの影響が長く残る（金融データではよくある）([mingze-gao.com][5])
* 各係数の **p値が十分小さいか** → その係数が統計的に意味があるか

### ④ Ljung-Box 2種類の検定の意味

1. **標準化残差そのもの**

   * 目的：平均モデル（ここでは定数 + GARCH）が十分か？
   * p値が小さい→まだ自己相関が残っている → AR を入れるなどを検討

2. **標準化残差の2乗**

   * 目的：分散モデル（GARCH）が ARCH 効果を取り切れているか？
   * ここで p値が小さい→GARCH(1,1) でもボラティリティのパターンを捉え切れていない
     → 次数アップや EGARCH / GJR-GARCH などを検討

この 2つを見ることで、

* **平均のモデリング**
* **分散のモデリング**

がそれぞれ十分かどうかを切り分けて考えられます。

### ⑤ 予測ボラティリティの解釈

```python
forecasts = result.forecast(horizon=5)
predicted_variance = forecasts.variance.iloc[-1]
predicted_volatility = np.sqrt(predicted_variance)
```

* `predicted_volatility` は「次の1日〜5日」の
  **予測ボラティリティ（標準偏差, %単位）** です。
* 例えば `h.1` が 1.2 なら、翌日の 1標準偏差レンジは
  「±1.2%」程度と解釈できます（※リターンを 100倍しているため）。

これを過去の `result.conditional_volatility` と一緒にプロットすると、

* 過去：実際のデータに対して GARCH がどのようにボラティリティを推定したか
* 将来：それを元に、近い将来のボラティリティをどう予測しているか

が視覚的に理解しやすくなります。

---

もし良ければ、次のステップとして：

* **mean='AR'** に変えて AR-GARCH にしてみる
* dist を `'StudentsT'` に変えて「ファットテール対応 GARCH」を試す

などを一緒にやってみましょう。「平均モデル」「分散モデル」「誤差分布」を組み合わせていくと、GARCH 系の理解が一気に深まります。

[1]: https://qiita.com/maskot1977/items/ce60b82ff51feaf5f8f0?utm_source=chatgpt.com "ChatGPTとPythonで学ぶ GARCH #arch"
[2]: https://media.readthedocs.org/pdf/arch/stable/arch.pdf?utm_source=chatgpt.com "arch Documentation"
[3]: https://arch.readthedocs.io/en/stable/univariate/introduction.html?utm_source=chatgpt.com "Introduction to ARCH Models - arch 7.2.0"
[4]: https://en.wikipedia.org/wiki/Autoregressive_conditional_heteroskedasticity?utm_source=chatgpt.com "Autoregressive conditional heteroskedasticity"
[5]: https://mingze-gao.com/posts/garch-estimation/?utm_source=chatgpt.com "GARCH Estimation"


<br>

# mean="AR"に換えた場合について

いいですね、ここで一気に「AR-GARCH」を押さえておくと、だいぶ実務寄りになります。

### 1. mean="Constant" と mean="AR" の違い

元コード（mean='Constant'）では、リターン (r_t) のモデルは

$$
r_t = \mu + \epsilon_t
$$

だけでした。
これを **mean='AR'** にすると、平均の部分が **自己回帰（AR）** になります：

$$
r_t = \mu + \varphi_1 r_{t-1} + \varphi_2 r_{t-2} + \cdots + \epsilon_t
$$

ARCH/GARCH 部分（分散のモデル）はそのままなので、

$$
\sigma_t^2 = \omega + \alpha_1 \epsilon_{t-1}^2 + \beta_1 \sigma_{t-1}^2
$$

と組み合わせると、**AR(p)–GARCH(1,1)** モデルになります。
`arch_model` では、

* `mean='AR'`
* `lags=1` なら AR(1)
* `lags=2` なら AR(2)
* `lags=[1,3]` のようにリストで「使いたいラグだけ指定」も可能

という指定方法になっています。

---

### 2. コードを mean="AR" に変える例

元のコードの GARCH 部分だけ差し替えると、こうなります。

```python
from arch import arch_model

# --- AR(1)-GARCH(1,1) モデルの指定 ---
# mean='AR' : 平均モデルを自己回帰(AR)にする
# lags=1    : AR(1)（1期前のリターンを使う）
am = arch_model(
    returns,
    mean='AR',
    lags=1,          # ★ここが追加
    vol='Garch',
    p=1, q=1,
    dist='Normal'
)

result = am.fit(update_freq=5)
print(result.summary())
```

このようにすると、サマリーには

* Mean Model のところに

  * `Const`（定数 μ）
  * `sp500_returns[1]` などの **AR係数**
* Volatility Model はこれまで通り

  * `omega`, `alpha[1]`, `beta[1]`

が並びます。

---

### 3. 何がうれしいのか（いつ mean='AR' を使うか）

**直感的な役割分担**はこうです：

* **平均モデル（mean='Constant' or 'AR'）**
  → リターンの「予測可能な成分」を捉える
* **分散モデル（GARCH）**
  → 「予測できないショックの大きさの自己相関（ボラティリティクラスタリング）」を捉える

もし mean='Constant' の状態で、

* 標準化残差 `std_resid` に Ljung-Box 検定をかけたとき **p値が小さい** → 残差に自己相関が残っている

  * ⇒ 「平均のモデリングが足りていない」サイン
  * ⇒ mean='AR', lags=1（または 2）などで **AR成分を入れてあげる**

と考えます。

逆に、

* mean='Constant' のままでも `std_resid` の Ljung-Box の p値が十分大きい
  → そもそもリターンにほぼ自己相関がない
  → 無理に mean='AR' にしなくてもよい（むしろベタな金融データではけっこうよくある）

というケースも多いです。

---

### 4. AR(1)-GARCH(1,1) の数式イメージ

`mean='AR', lags=1` の場合、モデルは：

$$
\begin{aligned}
r_t &= \mu + \varphi_1 r_{t-1} + \epsilon_t, \
\epsilon_t &= \sigma_t z_t,\quad z_t \sim \mathcal{N}(0,1), \
\sigma_t^2 &= \omega + \alpha_1 \epsilon_{t-1}^2 + \beta_1 \sigma_{t-1}^2.
\end{aligned}
$$

* $(\varphi_1)$ が **0 に近い** → 過去リターンからの予測力はほとんどない（効率的市場っぽい）
* $(\varphi_1)$ が **正で有意** → 「前日上がると、今日も少し上がりやすい」みたいなモメンタム
* $(\varphi_1)$ が **負で有意** → 「前日の動きが反転しやすい」ようなリバーサル

推定後は、`result.summary()` の Mean Model の行で

* `sp500_returns[1]` の `coef`（$= (\varphi_1)$）
* その `P>|t|`（p値）

を見て、「AR を入れる価値があったか」を確認します。

---

### 5. mean='AR' にしたときの診断の読み方

`std_resid` / `std_resid**2` に対する Ljung-Box は、AR-GARCH にしても同じように使えます。

1. **標準化残差 (`std_resid`) の Ljung-Box**

   * 目的：AR を入れたことで「平均のパターン」が取り切れているか？
   * AR を入れる前より p値が上がり、0.05を超えるようなら
     →「平均モデルが改善した」と判断しやすいです。

2. **標準化残差の二乗 (`std_resid**2`) の Ljung-Box**

   * 目的：GARCH 部分でボラティリティクラスタリングを取り切れているか？
   * ここは mean='Constant' のときと同じ読み方
     → p値が小さければ GARCH 次数やモデルの見直し。

---

### 6. lags をどう選ぶか？

実務的には、だいたい次のように決めます：

1. **ACF・PACF をざっと見る**

   * returns の ACF/PACF プロットを描いて、

     * ラグ1だけ大きい → AR(1)
     * ラグ1と2がそこそこ → AR(2)
2. **情報量基準（AIC/BIC）を見る**

   * `lags=1,2,3,...` を変えながらフィットして、AIC/BICが一番小さいものを選ぶ
3. **経済的な直感**

   * 日次株価リターン → そもそも AR がほぼゼロのことが多いので、無理に入れない
   * 低頻度のマクロ指標など → AR が効きやすいのでもう少し高次まで試す価値あり

`arch_model` の `lags` は
**「スカラーなら 1〜lags まで全部」**
**「リストなら指定したラグだけ」**
という仕様です。

---

### 7. まとめ：mean="AR" を使うときのストーリー

1. まずは mean='Constant', GARCH(1,1) でフィット
2. 標準化残差に自己相関が残っていたら
   → mean='AR', lags=1（必要なら2,3…）で AR-GARCH に変更
3. `result.summary()` で AR 係数の有意性を見る
4. Ljung-Box で `std_resid` の自己相関が消えているか確認
5. 問題なければ、その AR-GARCH モデルでボラティリティ予測へ

この流れをひと通り回せるようになると、

* 「平均のモデリング」と「ボラティリティのモデリング」を切り分けて考える
* AR 部分を足した方が良いケース／いらないケースを判断できる

という、より“中級者っぽい”立ち回りができるようになります。

---

もしよければ、次は：

* `mean='AR', lags=1` と `mean='Constant'` で Ljung-Box の結果や AIC/BIC を比較する
* ある銘柄（日次ではなく 1時間足など）で AR 部分が効いている例を一緒に見てみる

…という「実データ比較」もやってみましょうか？

<br>

## 実データ比較

いいですね、「実データで mean='Constant' と mean='AR' を比べる」までやると、一気に“使いこなしてる感”が出ます 😊

ここでは **「自分のPCでそのまま回せる比較ノート」** をイメージして説明します。

---

## 全体の流れ

1. 共通の前処理（S&P500 のリターンを作る）
2. モデル①：mean='Constant', GARCH(1,1)
3. モデル②：mean='AR', lags=1 の AR(1)-GARCH(1,1)
4. AIC/BIC と Ljung-Box の結果を横並びで比較
5. 結果をどう判断するか（何を見ればよいか）

> ※ 実際の数値は、あなたの環境で実行した結果に従ってください。
> ここでは「どういうコードを書き、何を見て、どう解釈するか」を軸に説明します。

---

## 1. 共通の前処理（S&P500 日次リターン）

前回とほぼ同じです。後でそのままコピペで使えるようにまとめます。

```python
import pandas as pd
import numpy as np
import yfinance as yf
from arch import arch_model
from statsmodels.stats.diagnostic import acorr_ljungbox
import matplotlib.pyplot as plt

# データ取得
data = yf.download('^GSPC', start='2010-01-01', end='2023-12-31')

# 日次リターン（%単位）
returns = 100 * data['Adj Close'].pct_change().dropna()
returns.name = 'sp500_returns'

# 確認用プロット（任意）
returns.plot(figsize=(12, 4), title='S&P 500 Daily Returns (%)')
plt.grid(True)
plt.show()
```

---

## 2. モデル①：mean='Constant', GARCH(1,1)

まずは「基準モデル」として、前回の mean='Constant' モデルをフィットします。

```python
# ========== モデル1: Constant + GARCH(1,1) ==========
am_const = arch_model(
    returns,
    mean='Constant',
    vol='Garch',
    p=1, q=1,
    dist='Normal'
)

res_const = am_const.fit(update_freq=5)
print(res_const.summary())
```

ここで見るポイントは：

* `omega`, `alpha[1]`, `beta[1]` の符号と有意性 (P>|z|)
* AIC/BIC（後で使うので取り出す）
* Ljung-Box で残差の自己相関が残っているか

診断コード：

```python
# 標準化残差
std_resid_const = res_const.std_resid.dropna()

# Ljung-Box (残差)
lb_const_resid = acorr_ljungbox(std_resid_const, lags=[1, 5, 10], return_df=True)

# Ljung-Box (残差^2)
lb_const_sq = acorr_ljungbox(std_resid_const**2, lags=[1, 5, 10], return_df=True)

print("=== Model 1: Constant + GARCH(1,1) ===")
print("AIC:", res_const.aic)
print("BIC:", res_const.bic)
print("\nLjung-Box on std_resid:")
print(lb_const_resid)
print("\nLjung-Box on std_resid^2:")
print(lb_const_sq)
```

---

## 3. モデル②：mean='AR', lags=1 → AR(1)-GARCH(1,1)

次に **平均に AR(1) を入れたモデル** を同じデータで推定します。

```python
# ========== モデル2: AR(1) + GARCH(1,1) ==========
am_ar1 = arch_model(
    returns,
    mean='AR',  # ★ここだけ違う
    lags=1,     # AR(1)
    vol='Garch',
    p=1, q=1,
    dist='Normal'
)

res_ar1 = am_ar1.fit(update_freq=5)
print(res_ar1.summary())
```

平均モデルは数式的にはこうなります：

$$
\begin{aligned}
r_t &= \mu + \varphi_1 r_{t-1} + \epsilon_t, \
\epsilon_t &= \sigma_t z_t,\quad z_t \sim N(0,1), \
\sigma_t^2 &= \omega + \alpha_1\epsilon_{t-1}^2 + \beta_1 \sigma_{t-1}^2
\end{aligned}
$$

サマリーで見るべきは：

* Mean Model のところの `sp500_returns[1]`（これが (\varphi_1)）

  * 係数の大きさ
  * `P>|t|` が小さいかどうか（有意かどうか）

診断コードも同様に：

```python
# 標準化残差
std_resid_ar1 = res_ar1.std_resid.dropna()

# Ljung-Box (残差)
lb_ar1_resid = acorr_ljungbox(std_resid_ar1, lags=[1, 5, 10], return_df=True)

# Ljung-Box (残差^2)
lb_ar1_sq = acorr_ljungbox(std_resid_ar1**2, lags=[1, 5, 10], return_df=True)

print("=== Model 2: AR(1) + GARCH(1,1) ===")
print("AIC:", res_ar1.aic)
print("BIC:", res_ar1.bic)
print("\nLjung-Box on std_resid:")
print(lb_ar1_resid)
print("\nLjung-Box on std_resid^2:")
print(lb_ar1_sq)
```

---

## 4. 結果の「横並び比較」

最後に、2モデルの指標を同じテーブルで見ると直感的です。

```python
summary_df = pd.DataFrame({
    'Model': ['Const+GARCH', 'AR(1)+GARCH'],
    'AIC': [res_const.aic, res_ar1.aic],
    'BIC': [res_const.bic, res_ar1.bic],
    'LB_resid_pval_lag10': [
        lb_const_resid['lb_pvalue'].iloc[-1],
        lb_ar1_resid['lb_pvalue'].iloc[-1]
    ],
    'LB_resid2_pval_lag10': [
        lb_const_sq['lb_pvalue'].iloc[-1],
        lb_ar1_sq['lb_pvalue'].iloc[-1]
    ]
})
print(summary_df)
```

上のような表が得られます。
実際に実行すると、例えば「こんな感じ」の結果になることが多いです（数値はイメージ）：

```text
          Model        AIC        BIC  LB_resid_pval_lag10  LB_resid2_pval_lag10
0  Const+GARCH  1234.50    1260.10          0.18                  0.42
1  AR(1)+GARCH  1233.90    1265.80          0.20                  0.40
```

典型的な S&P500 日次リターンだと：

* AR(1) の係数 (\varphi_1) は **かなり小さくて非有意** なことが多い
* AIC は少し良くなることもあるが、BIC（ペナルティが強い）は逆に悪くなることもある
* Ljung-Box の p値は、どちらのモデルでも十分大きくて「残差自己相関なし」と判定できることが多い

→ **結論:**
「このデータでは、わざわざ mean='AR' にしても大きな改善はなさそう。mean='Constant' でも十分。」
という判断になるケースが多いです。

逆に、**1時間足の為替データ** などでは：

* AR(1) が有意に負になって「直近の動きが反転しやすい」
* Ljung-Box も AR を入れた方がきれいに自己相関が消える

といったことがあり、その場合は

> AR(1)-GARCH の方が「平均のパターン」をきちんと説明してる

と判断できます。

---

## 5. 何を見てモデルを選ぶか（実務的な判断軸）

ざっくり言うと、

1. **AR係数の有意性**

   * `sp500_returns[1]` の P>|t| が 0.05 未満か？
   * 大きくて有意 → AR を入れる意味がある
   * 小さい・非有意 → 無理に入れなくてもよい

2. **情報量基準（AIC/BIC）**

   * AIC: 精度重視寄り（パラメータ数への罰則は弱め）
   * BIC: シンプルさ重視寄り（パラメータ数への罰則が強い）
   * 両方とも AR モデルの方が明確に小さい → AR を採用
   * ほとんど差がない or BIC が悪化 → シンプルな constant でもOK

3. **Ljung-Box の p値**

   * **標準化残差** の p値（自己相関）

     * AR を入れたことで p値が上がって「0.05超え」になった → 改善
   * **標準化残差²** の p値（残りの ARCH 効果）

     * ここは GARCH 部分の善し悪しなので、mean='AR' にしても大きくは変わらないことも多い

---

## まとめ

* **今回の比較コード**でやっていること：

  1. 同じリターン系列に対して
  2. Constant+GARCH と AR(1)+GARCH を両方フィットし
  3. AIC/BIC と Ljung-Box、AR係数の有意性で比較している

* **理解してほしいポイント**：

  * 「平均部分のモデル化」と「ボラティリティのモデル化」は役割が違う
  * mean='AR' にするのは、「平均にまだ予測可能な構造が残っている」ときだけでよい
  * S&P500 の日次リターンのような典型的な金融データでは、
    **mean='Constant' で十分なことも多い**（効率的市場仮説っぽい世界）

もしよければ、次は：

* 1時間足のFX（例：`'USDJPY=X'`）か、先物データで同じ比較をしてみて
  「どのデータだと AR が効いて、どのデータだと効かないか」を一緒に整理する

みたいなステップもやってみましょうか？





<br>


# dist を 'StudentsT' に変えて「ファットテール対応 GARCH」を試す

dist を `'StudentsT'` に変えると、

> 「**分散は GARCH で時変だけど、ショックの分布は正規ではなく“尻の重い t 分布”**」

というモデルになります。
金融リターンのように**ジャンプや極端値が多いデータ**では、かなり実務的に重要な選択肢です。

---

## 1. 正規分布 vs t分布（StudentsT）の考え方

GARCH モデルでは

$$
\epsilon_t = \sigma_t z_t
$$

の **(z_t) の分布**をどう仮定するか、が `dist` です。

* `dist='Normal'`
  → $(z_t \sim N(0,1))$（標準正規）

* `dist='StudentsT'`
  → $(z_t \sim t_\nu)$（自由度 $(\nu)$ の t 分布）

t分布は

* 真ん中は正規分布に似ているけど
* **裾が太い（fat-tail）**

  * 大きな値（= 大きなショック）の確率が正規より高い

ので、

> 「大暴落・急騰のような極端なリターンが、正規で想定するよりずっとよく起こる」

という金融の現実を反映しやすくなります。

`arch_model(..., dist='StudentsT')` にすると、

* 推定パラメータに `nu`（自由度） が増えます
* `nu` が小さいほど“裾が太い”

  * `nu ≈ 30` 以上 → ほぼ正規に近い
  * `nu ≈ 5` 〜 10 → かなり fat tail
  * `nu ≤ 2` だと理論上分散が無限になるので、通常はもっと大きい値になります

---

## 2. コード例：正規 vs t分布 の GARCH(1,1) を比較

### 2-1. 共通の前処理（S&P500 日次リターン）

前と同じですが、比較のために一度まとめます。

```python
import pandas as pd
import numpy as np
import yfinance as yf
from arch import arch_model
from statsmodels.stats.diagnostic import acorr_ljungbox
import matplotlib.pyplot as plt

# データ取得
data = yf.download('^GSPC', start='2010-01-01', end='2023-12-31')

# 日次リターン（%単位）
returns = 100 * data['Adj Close'].pct_change().dropna()
returns.name = 'sp500_returns'
```

---

### 2-2. モデル1：Normal-GARCH(1,1)

まずは基準となる「正規誤差の GARCH(1,1)」。

```python
# === Normal 誤差の GARCH(1,1) ===
am_norm = arch_model(
    returns,
    mean='Constant',
    vol='Garch',
    p=1, q=1,
    dist='Normal'   # ★ここが Normal
)
res_norm = am_norm.fit(update_freq=5)
print(res_norm.summary())
```

---

### 2-3. モデル2：StudentsT-GARCH(1,1)

次に `dist='StudentsT'` に変えたモデル。

```python
# === t分布誤差の GARCH(1,1) ===
am_t = arch_model(
    returns,
    mean='Constant',
    vol='Garch',
    p=1, q=1,
    dist='StudentsT'   # ★ここだけ変更
)
res_t = am_t.fit(update_freq=5)
print(res_t.summary())
```

`res_t.summary()` の Mean / Volatility に加えて、下の方に例えばこんな行が出ます（イメージです）:

```text
Distribution: StudentsT
...
nu      7.5    1.2   6.2   0.000  [5.2, 10.0]
```

* ここで `nu` が自由度
* p値が 0.05 よりかなり小さければ
  → 「正規ではなく t分布を使うべきだ」という統計的な根拠になる

---

## 3. Normal vs t分布 の診断比較

### 3-1. AIC/BIC と Ljung-Box

2モデルを横並びで比較してみましょう。

```python
from statsmodels.stats.diagnostic import acorr_ljungbox

def diag(res, name):
    std_resid = res.std_resid.dropna()
    lb_resid = acorr_ljungbox(std_resid, lags=[10], return_df=True)
    lb_sq    = acorr_ljungbox(std_resid**2, lags=[10], return_df=True)
    return {
        'Model': name,
        'AIC': res.aic,
        'BIC': res.bic,
        'LB_resid_pval_lag10': lb_resid['lb_pvalue'].iloc[-1],
        'LB_resid2_pval_lag10': lb_sq['lb_pvalue'].iloc[-1],
    }

diag_norm = diag(res_norm, 'Normal-GARCH(1,1)')
diag_t    = diag(res_t,    't-GARCH(1,1)')

summary_df = pd.DataFrame([diag_norm, diag_t])
print(summary_df)
```

典型的な金融データでは、よくこんな傾向になります（あくまで例）：

```text
              Model        AIC        BIC  LB_resid_pval_lag10  LB_resid2_pval_lag10
0  Normal-GARCH(1,1)   1234.5    1256.8          0.20                  0.03
1  t-GARCH(1,1)        1225.0    1251.0          0.22                  0.15
```

この場合の読み方：

* **AIC/BIC が小さい方がよい**
  → `t-GARCH` の方がよい（両方とも小さい）
* **LB_resid2_pval_lag10**

  * Normal-GARCH: 0.03 → まだ残差²に自己相関（ARCH効果）が残っているかも
  * t-GARCH: 0.15 → 0.05を超えていて OK

こうなっていれば、

> 「このデータでは、正規よりも t分布を使った方が GARCH モデルのあてはまりが良い」

と判断しやすくなります。

---

## 4. 「fat-tail 対応 GARCH」をどう理解すればいいか

### 4-1. 何が変わるのか？

* **分散の推定（σ_t²）** 自体は、Normal と t で大きく変わらないことも多いです
  → GARCH 部分は「時間的なボラティリティの構造」を見ているから
* 一方で、**極端なリターンの扱い**が変わります：

  * Normal-GARCH だと、大きな外れ値を「異常なデータ」として過剰に影響させてしまうことも
  * t-GARCH だと、もともと裾が太いので

    * 「極端値がそこそこ起こりやすい世界」を前提に推定
    * ⇒ **パラメータ推定がロバスト**になりやすい

### 4-2. 実務的なメリット

* オプション価格や VaR を計算するとき：

  * t-GARCH の方が「暴落時のリスク」を大きめに見積もることが多い
  * リスク管理の観点では、安全側に倒したいときに好まれる
* モデル診断上も

  * Normal-GARCH では標準化残差の尾部にまだ大きな歪み・尖度が残る
  * t-GARCH では残差分布がだいぶ「きれい」になることが多い

---

## 5. さらに一歩：標準化残差のヒストグラムと Q–Q プロット

Normal と t の違いを **見た目で理解**するために、標準化残差をプロットしてみると良いです。

```python
import scipy.stats as st

# 標準化残差
std_resid_norm = res_norm.std_resid.dropna()
std_resid_t    = res_t.std_resid.dropna()

fig, axes = plt.subplots(2, 2, figsize=(12, 8))

# (1) Normal-GARCH の残差ヒストグラム + 正規密度
ax = axes[0, 0]
std_resid_norm.hist(bins=50, density=True, alpha=0.6, ax=ax)
x = np.linspace(std_resid_norm.min(), std_resid_norm.max(), 200)
ax.plot(x, st.norm.pdf(x), 'r-', label='N(0,1)')
ax.set_title('Normal-GARCH std_resid (hist)')
ax.legend()
ax.grid(True)

# (2) t-GARCH の残差ヒストグラム + 正規密度
ax = axes[0, 1]
std_resid_t.hist(bins=50, density=True, alpha=0.6, ax=ax)
x = np.linspace(std_resid_t.min(), std_resid_t.max(), 200)
ax.plot(x, st.norm.pdf(x), 'r-', label='N(0,1)')
ax.set_title('t-GARCH std_resid (hist)')
ax.legend()
ax.grid(True)

# (3) Normal-GARCH の Q-Q プロット
ax = axes[1, 0]
st.probplot(std_resid_norm, dist="norm", plot=ax)
ax.set_title('Normal-GARCH Q-Q plot')

# (4) t-GARCH の Q-Q プロット
ax = axes[1, 1]
st.probplot(std_resid_t, dist="norm", plot=ax)
ax.set_title('t-GARCH Q-Q plot')

plt.tight_layout()
plt.show()
```

* Normal-GARCH の Q–Q プロットでは、両端（尾部）が直線から大きく外れやすい
* t-GARCH の方が、尾部の外れ具合が少しマシ（＝正規により近づく）になるケースが多い

→ これを見ると、「t分布でフィットした方が、残差はだいぶ“正規に近い形”にまで処理されているな」という感覚がつかみやすいです。

---

## 6. まとめ：`dist='StudentsT'` を使うときの判断フロー

1. まずは `dist='Normal'` で GARCH をフィット
2. 標準化残差の

   * ヒストグラム
   * Q–Q プロット
   * 歪度・尖度
   * Ljung-Box（残差²）
     を見て、「まだ尾部がおかしい／ARCH効果残ってそう」と感じたら
3. `dist='StudentsT'` に変えて再推定
4. AIC/BIC・残差診断・自由度 `nu` の有意性をチェック

   * AIC/BIC↓
   * `nu` の p値↓
   * 残差²の Ljung-Box の p値↑
     なら **t-GARCH 採用の理由がかなり強い**

---

ここまで押さえられると、

* 「平均：Constant vs AR」
* 「分布：Normal vs StudentsT」
* 「分散：GARCH の次数を変える」

といった**3つのレバーを自分で組み合わせて設計・診断**できるようになるので、かなり “中級 GARCH 使い” のレベルです。

次の一歩としては、

* `dist='StudentsT'` ＋ `mean='AR'` を一緒にした AR-GARCH-t
* 非対称モデル（`vol='Garch'` → GJR-GARCH / EGARCH に変える）

あたりをやると、実務のモデルにかなり近づきます。
興味があれば、GJR-GARCH か EGARCH のどちらかを、今回と同じスタイルで一緒にやってみましょうか？

<br>

## 非対称モデルについて


いいところに来ました！
ここを押さえると「GARCHをわかってる人」感が一気に出ます 😊

---

## 0. なぜ「非対称モデル」が必要か？

まず、ふつうの GARCH(1,1) では、

$$
\sigma_t^2 = \omega + \alpha \epsilon_{t-1}^2 + \beta \sigma_{t-1}^2
$$

という形で、**ショックの大きさ（$(\epsilon_{t-1}^2)$）だけ**を見ています。

* 昨日 +5% 上昇 → $(\epsilon_{t-1}^2)$ 大
* 昨日 -5% 下落 → $(\epsilon_{t-1}^2)$ 大

どちらも **同じだけ** 将来ボラティリティを上げる、と仮定しているわけです。

でも現実の株式市場では、

> 同じ 5% でも「下落」の方が、その後のボラティリティを大きく上げる

という **レバレッジ効果（asymmetric volatility / leverage effect）** がよく観測されます。

この「**マイナスのショックの方が効きが強い**」をモデル化するのが、GJR-GARCH や EGARCH といった **非対称 GARCH** です。

---

## 1. GJR-GARCH（しきい値付き GARCH）

### 1.1 数式イメージ

代表的なのが GJR-GARCH(1,1,1)：

$$
\sigma_t^2
= \omega + \alpha \epsilon_{t-1}^2 + \gamma \epsilon_{t-1}^2 I_{{\epsilon_{t-1}<0}} + \beta \sigma_{t-1}^2
$$

- $(I_{{\epsilon_{t-1}<0}})$ は「前期のショックがマイナスなら 1、そうでなければ 0」のダミー
- $(\gamma)$ が **レバレッジ効果の強さ**

  * $(\gamma > 0)$ なら「マイナスのショックの方が、ボラティリティをより強く押し上げる」

プラスのショック（$(\epsilon_{t-1}>0)$）のとき：
$$
\sigma_t^2 = \omega + \alpha \epsilon_{t-1}^2 + \beta \sigma_{t-1}^2
$$

マイナスのショック（$(\epsilon_{t-1}<0)$）のとき：
$$
\sigma_t^2 = \omega + (\alpha + \gamma)\epsilon_{t-1}^2 + \beta \sigma_{t-1}^2
$$

→ **下落時には係数が $(\alpha + \gamma)$ に増える**ので、「悪材料ほどボラティリティを高める」モデルになります。

### 1.2 Python（arch）での実装：`o=1` を付ける

`arch` ライブラリでは、

* `vol='GARCH'`
* `p`：GARCH ラグ数
* `q`：ARCH ラグ数
* `o`：**非対称（しきい値）ラグ数**

を指定することで **GJR-GARCH** を実装できます。

#### 前処理（共通）

```python
import pandas as pd
import numpy as np
import yfinance as yf
from arch import arch_model
from statsmodels.stats.diagnostic import acorr_ljungbox
import matplotlib.pyplot as plt

# データ取得
data = yf.download('^GSPC', start='2010-01-01', end='2023-12-31')

# 日次リターン（%単位）
returns = 100 * data['Adj Close'].pct_change().dropna()
returns.name = 'sp500_returns'
```

#### GJR-GARCH(1,1,1) + t分布 誤差

```python
# GJR-GARCH(1,1,1) モデル
# p=1: GARCH項, q=1: ARCH項, o=1: 非対称(負のショック用)項
gjr = arch_model(
    returns,
    mean='Constant',
    vol='GARCH',   # GARCH プロセス
    p=1,
    o=1,           # ★ここが GJR-GARCH のポイント
    q=1,
    dist='StudentsT'  # ファットテール対応
)
res_gjr = gjr.fit(update_freq=5)
print(res_gjr.summary())
```

サマリーの Volatility 部分に、例えばこんな項目が出てきます（イメージ）：

* `omega`
* `alpha[1]`
* `gamma[1]`  ← これが (\gamma)（レバレッジ効果）
* `beta[1]`

見るべきポイント：

* `gamma[1]` の係数が

  * 正か？（マイナスのショックでボラティリティがより増える）
  * P>|z| が 0.05 より十分小さいか？（統計的に有意か？）
* `alpha[1] + gamma[1] + beta[1]` が 1 に近いか？

  * GARCH と同様、ボラティリティ持続性の指標

---

## 2. EGARCH（Exponential GARCH）

### 2.1 特徴と数式イメージ

EGARCH(1,1) は、**分散ではなく「log 分散」を直接モデル化**します。

典型的な EGARCH(1,1) の形：

$$
\log \sigma_t^2
= \omega + \beta \log \sigma_{t-1}^2 +  \alpha \left( |z_{t-1}| - E|z_{t-1}| \right) + \gamma z_{t-1}
$$


- $(z_{t-1} = \epsilon_{t-1} / \sigma_{t-1})$（標準化残差）
- $(|z_{t-1}|)$ の項 … **ショックの絶対値**（大きさ）への反応
- $(z_{t-1})$ の項 … **ショックの符号（正/負）**への反応（非対称性）

  * $(\gamma < 0)$ だと、負のショック（$(z_{t-1}<0)$）のとき、対数分散がより大きく増える

EGARCH の利点： 

1. **log を取っているので $(\sigma_t^2>0)$ を自然に保証**（パラメータに非負制約をかけなくてよい）
2. 非対称成分が「きれいな形」で入る（符号と大きさを分けて扱える）

### 2.2 Python（arch）での実装：`vol='EGARCH'`

`arch_model` で `vol='EGARCH'` を指定すれば EGARCH が使えます。

```python
# EGARCH(1,1) モデル（非対称ショック付き）
egarch = arch_model(
    returns,
    mean='Constant',
    vol='EGARCH',
    p=1,  # GARCH ラグ
    o=1,  # 非対称（レバレッジ）項のラグ数
    q=1,  # ARCH ラグ
    dist='StudentsT'
)
res_egarch = egarch.fit(update_freq=5)
print(res_egarch.summary())
```

サマリーでは、Volatility のところに

* `omega`
* `alpha[1]`
* `gamma[1]`（レバレッジ効果）
* `beta[1]`

などが表示されます。

解釈の仕方：

* `gamma[1]` が負で有意 →
  「マイナスの標準化ショック（悪材料）のとき、log σ² がより伸びる」
  → 負のショックの方が将来ボラティリティを強く押し上げる（レバレッジ効果）

---

## 3. 対称 GARCH vs 非対称 GJR / EGARCH の比較コード

「どれが良さそうか」を見るために、AIC/BIC と Ljung-Box を並べるとわかりやすいです。

```python
from statsmodels.stats.diagnostic import acorr_ljungbox

def diag(res, name):
    std_resid = res.std_resid.dropna()
    lb_resid = acorr_ljungbox(std_resid, lags=[10], return_df=True)
    lb_sq    = acorr_ljungbox(std_resid**2, lags=[10], return_df=True)
    return {
        'Model': name,
        'AIC': res.aic,
        'BIC': res.bic,
        'LB_resid_pval_lag10': lb_resid['lb_pvalue'].iloc[-1],
        'LB_resid2_pval_lag10': lb_sq['lb_pvalue'].iloc[-1],
    }

# 1. 対称 GARCH(1,1)
garch = arch_model(returns, mean='Constant', vol='GARCH',
                   p=1, q=1, dist='StudentsT')
res_garch = garch.fit(disp='off')

# 2. GJR-GARCH(1,1,1)
gjr = arch_model(returns, mean='Constant', vol='GARCH',
                 p=1, o=1, q=1, dist='StudentsT')
res_gjr = gjr.fit(disp='off')

# 3. EGARCH(1,1)
egarch = arch_model(returns, mean='Constant', vol='EGARCH',
                    p=1, o=1, q=1, dist='StudentsT')
res_egarch = egarch.fit(disp='off')

summary_df = pd.DataFrame([
    diag(res_garch, 'GARCH(1,1)'),
    diag(res_gjr,   'GJR-GARCH(1,1,1)'),
    diag(res_egarch,'EGARCH(1,1)')
])
print(summary_df)
```

よくあるパターン（イメージ）：

```text
             Model        AIC        BIC  LB_resid_pval_lag10  LB_resid2_pval_lag10
0        GARCH(1,1)  1234.5    1256.8            0.20                   0.04
1  GJR-GARCH(1,1,1)  1228.0    1256.0            0.25                   0.18
2       EGARCH(1,1)  1227.5    1255.5            0.26                   0.20
```

こんな感じなら：

* AIC/BIC：EGARCH, GJR-GARCH の方が小さい
* 残差²の Ljung-Box：非対称モデルの方が p値が大きく、「残りの ARCH 効果」が減っている

→ **非対称モデルの方が、このデータのボラティリティの振る舞いをうまく捉えている**
と判断できます。

---

## 4. どういうデータで非対称モデルが効きやすいか？

* 株式・株価指数・クレジットスプレッドなど

  * 「悪材料が出ると一気に下落＆ボラ急騰」というレバレッジ効果が強い
  * GJR-GARCH / EGARCH の `gamma` が有意になることが多い
* 為替や商品先物など

  * シリーズによっては「上も下も対称」なものもある → 対称 GARCH で十分なことも

なので実務的には、

1. まず **対称 GARCH(1,1)** でフィット
2. 残差診断・ニュースインパクトの非対称性が気になれば
3. GJR-GARCH, EGARCH を試して

   * AIC/BIC
   * Ljung-Box
   * `gamma` の符号と有意性

を見て、**非対称モデルを採用するか決める**、という流れが多いです。

---

もし次のステップとして余力があれば、

* 同じデータで GJR-GARCH と EGARCH の `gamma` の符号・大きさ・有意性を比べる
* `mean='AR'` も組み合わせて AR(1)-EGARCH, AR(1)-GJR-GARCH をフィットして AIC/BIC を比較する

というところまでやると、「平均・分散・分布・非対称性」を全部コントロールできる状態になります 👍

そこまでやってみたい場合は、「S&P500 で GJR と EGARCH を実際に比較する notebook っぽいコードを一式ください」と言ってくれれば、そこまでまとめてお渡しします。





<br>

# 「平均・分散・分布・非対称性」を全部コントロールできる状態になるために、S&P500 で GJR と EGARCH を実際に比較する notebook っぽいコード

ここまで来たら、もう「GARCH職人」ですね 😊
S&P500 を題材にして、

* **平均**：Constant vs AR(1)
* **分散モデル**：対称 GARCH vs 非対称 GJR-GARCH / EGARCH
* **分布**：Normal vs StudentsT（t分布）
* **非対称性**：有り / なし

を全部切り替えながら比較できる「ノートブック一式」を用意します。

---

## 全体像

下のコードは、Jupyter でそのまま上から順に実行できる構成になっています：

1. セットアップ & データ取得
2. 共通ユーティリティ（`fit_and_diagnose` 関数）
3. 4つのモデルをフィット

   * M1: AR(1) + GARCH(1,1) + Normal
   * M2: AR(1) + GARCH(1,1) + StudentsT
   * M3: AR(1) + **GJR-GARCH(1,1,1)** + StudentsT
   * M4: AR(1) + **EGARCH(1,1)** + StudentsT
4. モデル比較テーブル（AIC/BIC, Ljung-Box, 自由度 `nu`, 非対称パラメータ `gamma`）
5. 条件付きボラティリティの可視化

---

## 1. セットアップ & データ取得

```python
# ===========================
# セットアップ & データ取得
# ===========================
# ※ 初回のみ:
# !pip install arch yfinance

import pandas as pd
import numpy as np
import yfinance as yf
from arch import arch_model
from statsmodels.stats.diagnostic import acorr_ljungbox
import matplotlib.pyplot as plt
import scipy.stats as st

plt.style.use("default")  # お好みで

# --- S&P 500 (^GSPC) の日次データ取得 ---
data = yf.download('^GSPC', start='2010-01-01', end='2023-12-31')

# --- 日次リターン（%単位） ---
# pct_change() は 0.01 = 1% のような小さい値→これを 100倍して「%リターン」にするのが
# GARCH 推定を安定させる実務上の定石の一つです。
returns = 100 * data['Adj Close'].pct_change().dropna()
returns.name = 'sp500_returns'

# 確認用プロット（任意）
returns.plot(figsize=(12, 4), title='S&P 500 Daily Returns (%)')
plt.grid(True)
plt.show()
```

---

## 2. 共通ユーティリティ関数

1つの関数 `fit_and_diagnose` で、

* モデル構造（mean / vol / p,o,q / dist）を指定
* フィットして
* * AIC / BIC
  * Ljung-Box（残差 / 残差²）
  * `nu`（StudentsT の自由度）
  * 非対称パラメータ `gamma`
* をまとめて返します。

```python
# ===========================
# 共通ユーティリティ関数
# ===========================
from typing import Optional, Dict


def fit_and_diagnose(
    y: pd.Series,
    model_name: str,
    mean: str = "AR",      # "Constant" or "AR"
    lags: int = 1,         # mean="AR" のときの AR次数
    vol: str = "GARCH",    # "GARCH" or "EGARCH"
    p: int = 1,
    o: int = 0,
    q: int = 1,
    dist: str = "Normal"   # "Normal" or "StudentsT"
) -> Dict:
    """
    指定した構造の GARCH 系モデルをフィットし、
    結果オブジェクトと 診断指標をまとめて返すヘルパー。
    """
    # --- モデル定義 ---
    am = arch_model(
        y,
        mean=mean,
        lags=lags if mean == "AR" else None,
        vol=vol,
        p=p,
        o=o,
        q=q,
        dist=dist
    )

    # --- フィット ---
    res = am.fit(disp="off")  # ノイズを抑える

    # --- 標準化残差 ---
    std_resid = res.std_resid.dropna()

    # --- Ljung-Box 検定（ラグ10まで） ---
    lb_resid = acorr_ljungbox(std_resid, lags=[10], return_df=True)
    lb_resid2 = acorr_ljungbox(std_resid**2, lags=[10], return_df=True)

    # --- t分布の自由度（StudentsT の場合だけ） ---
    nu = None
    if dist.lower().startswith("student"):
        if "nu" in res.params.index:
            nu = res.params["nu"]

    # --- 非対称性パラメータ（GJRやEGARCHの gamma） ---
    gamma = None
    # GJR-GARCH: vol="GARCH", o>0 で gamma[1], gamma[2], ...
    for param_name in res.params.index:
        if param_name.lower().startswith("gamma"):
            gamma = res.params[param_name]
            break

    # --- 結果まとめ ---
    info = {
        "model_name": model_name,
        "result": res,
        "AIC": res.aic,
        "BIC": res.bic,
        "LB_resid_pval_lag10": lb_resid["lb_pvalue"].iloc[-1],
        "LB_resid2_pval_lag10": lb_resid2["lb_pvalue"].iloc[-1],
        "nu": nu,
        "gamma": gamma,
    }
    return info
```

---

## 3. 4つのモデルをフィットして比較

### モデル設計

* **M1**: AR(1) + GARCH(1,1) + Normal
* **M2**: AR(1) + GARCH(1,1) + StudentsT
* **M3**: AR(1) + GJR-GARCH(1,1,1) + StudentsT

  * `vol="GARCH", o=1` が GJR-GARCH（非対称 GARCH）
* **M4**: AR(1) + EGARCH(1,1) + StudentsT

  * `vol="EGARCH", o=1` が非対称 EGARCH

```python
# ===========================
# 4つのモデルをフィット
# ===========================
models = []

# M1: AR(1) + GARCH(1,1) + Normal
m1 = fit_and_diagnose(
    returns,
    model_name="M1: AR(1)-GARCH(1,1)-Normal",
    mean="AR", lags=1,
    vol="GARCH", p=1, o=0, q=1,
    dist="Normal"
)
models.append(m1)

# M2: AR(1) + GARCH(1,1) + StudentsT
m2 = fit_and_diagnose(
    returns,
    model_name="M2: AR(1)-GARCH(1,1)-t",
    mean="AR", lags=1,
    vol="GARCH", p=1, o=0, q=1,
    dist="StudentsT"
)
models.append(m2)

# M3: AR(1) + GJR-GARCH(1,1,1) + StudentsT
#     → 非対称項 o=1 を追加（負のショックへの反応を別パラメータで表現）
m3 = fit_and_diagnose(
    returns,
    model_name="M3: AR(1)-GJR-GARCH(1,1,1)-t",
    mean="AR", lags=1,
    vol="GARCH", p=1, o=1, q=1,
    dist="StudentsT"
)
models.append(m3)

# M4: AR(1) + EGARCH(1,1) + StudentsT
#     → vol='EGARCH', p=1, o=1, q=1 で非対称EGARCH
m4 = fit_and_diagnose(
    returns,
    model_name="M4: AR(1)-EGARCH(1,1)-t",
    mean="AR", lags=1,
    vol="EGARCH", p=1, o=1, q=1,
    dist="StudentsT"
)
models.append(m4)
```

---

## 4. モデル比較テーブルの作成と解説

### 4-1. 指標一覧表

```python
# ===========================
# モデル比較テーブル
# ===========================
summary_df = pd.DataFrame([
    {
        "Model": m["model_name"],
        "AIC": m["AIC"],
        "BIC": m["BIC"],
        "LB_resid_pval_lag10": m["LB_resid_pval_lag10"],
        "LB_resid2_pval_lag10": m["LB_resid2_pval_lag10"],
        "nu (if t)": m["nu"],
        "gamma (asymmetry)": m["gamma"],
    }
    for m in models
])

pd.set_option("display.float_format", "{:.4f}".format)
summary_df
```

### 4-2. 表の読み方（口頭解説）

このテーブルから、次のような視点でモデルを比べます：

1. **AIC / BIC**

   * 小さいほど「よい」モデル（データに対して当てはまりがよく、かつ過剰パラメータでない）
   * M1 vs M2 → **t分布で改善しているか？**
   * M2 vs M3 / M4 → **非対称モデルでさらに改善しているか？**

2. **LB_resid_pval_lag10**（標準化残差の Ljung-Box）

   * 0.05 より大きい → 残差に自己相関は残っていない（平均モデルはOK）

3. **LB_resid2_pval_lag10**（標準化残差²の Ljung-Box）

   * 0.05 より大きい → 残りの ARCH 効果はほぼない（分散モデルはOK）
   * M1 では 0.05 未満だが、M2, M3, M4 で 0.05 を超える…などなら
     → それだけ「うまくボラティリティを説明できている」モデル

4. **nu (if t)**（自由度）

   * t分布を使っているモデル（M2, M3, M4）でのみ値が入る
   * 小さいほど fat-tail（厚い裾）
   * `nu` の推定値と、その p値（`res.params` の summary で確認）も見る
     → 正規分布からの乖離が有意かどうか

5. **gamma (asymmetry)**（非対称パラメータ）

   * M3（GJR-GARCH） & M4（EGARCH）でのみ値が入るはず
   * 正負 & p値で解釈

     * GJR-GARCH: `gamma > 0` かつ有意
       → 負のショックのときの方がボラティリティがより増える
     * EGARCH: 一般に `gamma < 0` が「負のショックでボラ増大」になるように定義されることが多い
       → 公式の符号はモデルの実装に依存するので、ドキュメントや論文の符号慣習も参照（ただ、`gamma` の有意性と符号は「非対称性の存在」を示す）

---

## 5. 条件付きボラティリティの可視化

最後に、いくつかのモデルで **条件付きボラティリティ $(\sigma_t)$** を比較します。

```python
# ===========================
# 条件付きボラティリティの比較プロット
# ===========================

fig, ax = plt.subplots(2, 1, figsize=(12, 8), sharex=True)

# 実際のリターン
returns.plot(ax=ax[0])
ax[0].set_title("S&P 500 Daily Returns (%)")
ax[0].grid(True)

# 各モデルの条件付きボラティリティ
for m in models:
    res = m["result"]
    vol = res.conditional_volatility  # σ_t（%リターンの標準偏差）
    ax[1].plot(vol, label=m["model_name"])

ax[1].set_title("Conditional Volatility (σ_t) from Each Model")
ax[1].set_ylabel("Volatility (% per day)")
ax[1].grid(True)
ax[1].legend(loc="upper right")

plt.tight_layout()
plt.show()
```

### このグラフの見方

* 上段：実際のリターン（大きく上下している期間 ≒ ボラが高い期間）
* 下段：各モデルが推定した条件付きボラティリティの系列

  * Normal-GARCH（M1）：暴落局面でのボラのピークがやや違うかも
  * t-GARCH（M2）：極端値への耐性があり、推定が少し滑らかになることも
  * GJR / EGARCH（M3, M4）：

    * 特に「急落があった直後」にボラティリティがより強く立ち上がる傾向があれば、
      → **負のショックへの非対称な反応**をうまく捉えている証拠

---

## 6. このノートで「何ができるようになるか」

この一式で、次ができるようになります：

1. **平均（mean）**

   * `mean="Constant"` / `"AR"` と `lags` を変えて
     → リターンに自己相関があるか & それをどこまで説明するかをコントロール

2. **分散モデル（volatility dynamics）**

   * `vol="GARCH"` vs `"EGARCH"`
   * `p, q` を変えて
     → ボラティリティクラスタリングの強さ、持続性の表現力をコントロール

3. **分布（dist）**

   * `dist="Normal"` vs `"StudentsT"`
     → 極端値・ジャンプの扱いをコントロール（fat-tail 対応）

4. **非対称性（レバレッジ効果）**

   * `o=0`（対称） vs `o=1`（非対称）
   * `gamma` の符号・有意性を見る
     → 「下落の方がボラを上げやすい」現象をモデルに入れる / 入れないをコントロール

---

## 7. 次のステップ案

ここまで来ると、もうかなり中級以上です。
もしさらに深めるなら：

* **ニュースインパクトカーブ**（News Impact Curve）を描いて、

  * 同じ大きさの +ショック / -ショック が (\sigma_t^2) にどう影響するかを可視化
* 1時間足の FX や先物データに同じコードを適用して、

  * 「どの市場がより非対称なのか？」
  * 「どの頻度で AR 成分が効くのか？」

などを比較してみると、かなり実務的な感覚もついてきます。

もし「ニュースインパクトカーブを描くコードも欲しい」ですとか、「FX データで同じ比較をしたい」といった要望があれば、そのパターンも notebook 風に一式用意します！
